In [1]:
%load_ext autoreload
%autoreload 2
%load_ext watermark
%watermark -a "Kenneth Brezinski" -v

Author: Kenneth Brezinski

Python implementation: CPython
Python version       : 3.9.10
IPython version      : 8.0.1



In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_ww = pd.read_csv('data/WW-measure.csv')
df_mob = pd.read_csv('data/mobility.csv')

In [ ]:
df_ww.tail()

In [ ]:
## need to remove NaNs from date column
## sampleID needs to turn into datetime

In [ ]:
parsed_date = pd.to_datetime(df_ww['sampleID'].apply(lambda d: d.split('_')[1])).rename('date')
df_ww = pd.concat([parsed_date, df_ww[['uWWMeasureID', 'value']]], axis=1)

In [ ]:
# test example on Ontario WW
df_ww_ON = df_ww[df_ww['uWWMeasureID'].apply(lambda x: x.endswith('TAB'))]
df_ww_ON.head(2)

In [ ]:
# test example on Ontario mobility
df_ON = df_mob[df_mob['location_key'].apply(lambda x: x.startswith('CA_ON') if isinstance(x, str) else False)]
df_ON.head(2)

In [ ]:
df_ON.loc[:, 'date'] = pd.to_datetime(df_ON['date'], infer_datetime_format=True)

In [ ]:
df_merge = pd.merge(df_ON, df_ww_ON, how='outer', on='date').set_index('date')
df_merge.head(10)

In [ ]:
tmp = df_merge[df_merge['uWWMeasureID'].apply(lambda i: i.endswith('N1TAB') if isinstance(i, str) else False)]
tmp['value'].plot()
tmp['mobility_transit_stations'].plot()